### SImulating data

In this step we simulate some data in the following form:

$y_i = mx_i + b + \epsilon_i$

Where $\epsilon_i$ is in the form:

$\mathcal{N}(0, \sigma^2)$

For this case, we defined the following:

$m=5 \\ b= 2 \\ \sigma^2 = 9$

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

np.random.seed(100)

seed = [1,1,1,1,1,1,1]

m = 5
b = 2

std = 3

x = np.arange(0,11,1)
X = [x, x**2, x**3, x**4, x**5, x**6]

xForPred = np.arange(0,10.1,.1)
XForPred = [xForPred, xForPred**2, xForPred**3, xForPred**4, xForPred**5, xForPred**6]


epsilon = np.random.normal(0, std, (len(x))) 

y = m * x + b + epsilon

plt.scatter(x,y)

plt.show()

### Linear regression

In this step we do a Linear Regression with the form:

$y = mx + b$

By minimizing the error function:

$\sum_{i=1}^n(y_i - mx_i + b)^2$

In [ ]:
from scipy.optimize import minimize

def l_function(thetas):
    return sum((y - np.dot(thetas[0:-1],X[0:1]) - thetas[-1])**2)

res = minimize(l_function, seed[0:2], method='BFGS')

ypred = res.x[0] * x + res.x[1] 
plt.scatter(x,y)
plt.plot(ypred)
plt.show()

### Hypothesis

In this part we propose some hypotesis:

$H_0^a: y_i = \theta_1 x_1^i + \theta_2 (x_1^i)^2 + \theta_0 $

In [ ]:
def ha_function(thetas):
    return sum((y - np.dot(thetas[0:-1],X[0:2]) - thetas[-1])**2)

resHa = minimize(ha_function, seed[0:3], method='BFGS')

ypredHa = np.dot(resHa.x[0:-1],XForPred[0:2]) + resHa.x[-1]
plt.scatter(x,y)
plt.plot(xForPred, ypredHa)
plt.show()

$H_0^b: y_i = \theta_1 x_1^i + \theta_2 (x_1^i)^2 + \theta_3 (x_1^i)^3 + \theta_0$

In [ ]:
def hb_function(thetas):
    return sum((y - np.dot(thetas[0:-1],X[0:3]) - thetas[-1])**2)

resHb = minimize(hb_function, seed[0:4], method='BFGS')

ypredHb = np.dot(resHb.x[0:-1],XForPred[0:3]) + resHb.x[-1]
plt.scatter(x,y)
plt.plot(xForPred, ypredHb)
plt.show()

$H_0^c: y_i = \theta_1 x_1^i + \theta_2 (x_1^i)^2 + \theta_3 (x_1^i)^3 +\theta_4 (x_1^i)^4 +\theta_5 (x_1^i)^5 +\theta_6 (x_1^i)^6 +\theta_7 (x_1^i)^7 + \theta_0$

In this Hypotesis we can see that there is a lot of overfitting:

In [ ]:

def hc_function(thetas):
    return sum((y - np.dot(thetas[0:-1],X) - thetas[-1])**2)

resHc = minimize(hc_function, seed[0:7], method='BFGS')

xForPred = np.arange(0,10.1,.1)

ypredHc = np.dot(resHc.x[0:-1],XForPred) + resHc.x[-1]
plt.scatter(x,y)
plt.plot(xForPred, ypredHc)
plt.show()

### Ridge regularization 

To avoid overfitting, we are going to regularize the last function by using Ridge regularization, this takes the form of:

$\sum(y_i - m_{\theta}(x_i))^2 + \lambda (\theta_1^2 + \theta_1^2 + .... + \theta_n^2)$

In [ ]:
lambd = 100
# Lambda por cada uno de los thetas^2
def ridge_function(thetas):
    return sum((y - np.dot(thetas[0:-1],X) - thetas[-1])**2) + lambd*(sum(thetas**2))

resRidge = minimize(ridge_function, seed, method='BFGS')

ypredRidge = np.dot(resRidge.x[0:-1],XForPred) + resRidge.x[-1]
plt.scatter(x,y)
plt.plot(xForPred, ypredRidge)
plt.show()

### Regularizacion LASSO

To avoid overfitting, we are going to regularize the last function by using Lasso regularization, this takes the form of:

$\sum(y_i - m_{\theta}(x_i))^2 + \lambda (|\theta_1| + |\theta_1| + .... + |\theta_n|)$

In [ ]:
lambd = 100

def lasso_function(thetas):
    return sum((y - np.dot(thetas[0:-1],X) - thetas[-1])**2) + lambd * sum(abs(thetas))

resLasso = minimize(lasso_function, seed, method='BFGS')

ypredLasso = np.dot(resLasso.x[0:-1],XForPred) + resLasso.x[-1]
plt.scatter(x,y)
plt.plot(xForPred,ypredLasso)
plt.show()


### Cross validation

In order to choose the lambda value we use the following process called k-fold cross validation:

- Split the entire data randomly into k folds (value of k shouldn’t be too small or too high, ideally we choose 5 to 10 depending on the data size). The higher value of K leads to less biased model (but large variance might lead to overfit), where as the lower value of K is similar to the train-test split approach we saw before.
- Then fit the model using the K — 1 (K minus 1) folds and validate the model using the remaining Kth fold. Note down the scores/errors.
- Repeat this process until every K-fold serve as the test set. Then take the average of your recorded scores. That will be the performance metric for the model.